**Description**: save all the stuff sent to the OpenAI API by LangChain as JSONs or in a
dict. TODO: save outputs. Main wrinkle is handling streamed outputs.

**Setup**: Install these packages

```zsh
python -m pip install langchain langchain-openai
```

In [1]:
from contextlib import contextmanager
from datetime import datetime
from functools import partial, wraps
import json
import os
from typing import Any, Callable, Generator, MutableMapping

from langchain_openai import ChatOpenAI
from rich import print

# Export these

Copy these functions somewhere.

Could instead patch the class instead of the object so that one patch globally patches
all. But I'm not totally sure that'd work. I think I'd have to get
`client.create.__class__` and then patch its `create` method. For now, going to stick to
the more conservative instance patch.

In [2]:
@contextmanager
def _monkeypatch_instance_method(obj: Any, method_name: str, new_method: Callable):
    original_method = getattr(obj, method_name)
    # Need to use __get__ when patching instance methods
    # https://stackoverflow.com/a/28127947/18758987
    try:
        setattr(obj, method_name, new_method.__get__(obj, obj.__class__))
        yield
    finally:
        setattr(obj, method_name, original_method.__get__(obj, obj.__class__))


@contextmanager
def _run_method_with_side_effect(
    obj: Any, method_name: str, side_effect: Callable, yielded_obj=None
):
    original_method = getattr(obj, method_name)

    @wraps(original_method)
    def new_method(self, *args, **kwargs):
        side_effect(*args, **kwargs)
        return original_method(*args, **kwargs)

    with _monkeypatch_instance_method(obj, method_name, new_method):
        yield yielded_obj

In [3]:
def _write_json(jsons_dir: str, *args, **kwargs):
    # TODO: figure out what to do w/ args. For now, ignore them b/c the OpenAI client
    # create method requires that all arguments are named
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")
    file_path = os.path.join(jsons_dir, f"{current_time}.json")
    with open(file_path, mode="w") as file:
        json.dump(kwargs, file, indent=4)


@contextmanager
def save_chat_create_inputs_as_jsons(client_with_create_method: Any, jsons_dir: str):
    """
    In this context, save the inputs sent to some API through
    `client_with_create_method.create` in `jsons_dir`.

    Parameters
    ----------
    client_with_create_method : Any
        some object with a `create` method, e.g.,
        `langchain_openai.ChatOpenAI().client`. Its inputs will be saved as JSONs
        whenever this method is called
    jsons_dir : str
        directory where your JSONs will get saved. The file names are timestamps

    Example
    -------
    ::

        from langchain_openai import ChatOpenAI

        llm = ChatOpenAI()
        jsons_dir = "temp"  # make this directory yourself

        with save_chat_create_inputs_as_jsons(llm.client, jsons_dir):
            # Note that the llm.client object is modified in this context, so any code
            # that uses the llm will end up saving a JSON.
            response = llm.invoke("how can langsmith help with testing?")

        # Then look at the json in ./jsons_dir

    Note
    ----
    You probably only need the last JSON that's saved in `jsons_dir` b/c it'll contain
    the chat history.
    """
    side_effect = partial(_write_json, jsons_dir)
    with _run_method_with_side_effect(client_with_create_method, "create", side_effect):
        yield

Here's a non-eager way which just stores the inputs in a dict. This is better if you
wanna control where and when to store the data. That way, you avoid JSON-writing
overhead during the main application

In [4]:
def _update_dict(
    timestamp_to_kwargs: MutableMapping[str, dict[str, Any]], *args, **kwargs
):
    # TODO: figure out what to do w/ args. For now, ignore them b/c the OpenAI client
    # create method requires that all arguments are named
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")
    timestamp_to_kwargs[current_time] = kwargs


@contextmanager
def save_chat_create_inputs_as_dict(
    client_with_create_method: Any,
    existing_store: MutableMapping[str, dict[str, Any]] | None = None,
):
    """
    In this context, save the inputs sent to some API through
    `client_with_create_method.create` in a dictionary.

    Parameters
    ----------
    client_with_create_method : Any
        some object with a `create` method, e.g.,
        `langchain_openai.ChatOpenAI().client`. Its inputs will be saved in a dictionary
        whenever this method is called
    existing_store : MutableMapping[str, dict[str, Any]], optional
        an existing dictionary which you'd like to add to. Keys are timestamp strings.
        By default, a new dictionary will be created

    Example
    -------
    ::

        from langchain_openai import ChatOpenAI

        llm = ChatOpenAI()

        with save_chat_create_inputs_as_dict(llm.client) as timestamp_to_kwargs:
            # Note that the llm.client object is modified in this context, so any code
            # that uses the llm will end up saving its inputs in timestamp_to_kwargs.
            response = llm.invoke("how can langsmith help with testing?")

        print(timestamp_to_kwargs)

    Note
    ----
    You probably only need the last key-value that's saved in `timestamp_to_kwargs` b/c
    it'll contain the chat history.
    """
    if existing_store is None:
        timestamp_to_kwargs: MutableMapping[str, dict[str, Any]] = dict()
    else:
        timestamp_to_kwargs = existing_store
    side_effect = partial(_update_dict, timestamp_to_kwargs)
    with _run_method_with_side_effect(
        client_with_create_method,
        "create",
        side_effect,
        yielded_obj=timestamp_to_kwargs,
    ) as timestamp_to_kwargs:
        yield timestamp_to_kwargs

# Minimal demo

In [5]:
llm = ChatOpenAI()
jsons_dir = "temp"  # make this dir on your own pls

In [6]:
with save_chat_create_inputs_as_jsons(llm.client, jsons_dir):
    response = llm.invoke("Why am I addicted to monkey-patching in Python?")
print(response.content)

Monkey-patching in Python can be addictive for several reasons:

1. Instant gratification: Monkey-patching allows you to quickly and easily modify the behavior of existing code 
without having to go through the formal process of submitting a patch or waiting for a new release. This instant 
gratification can be very satisfying and addictive.

2. Flexibility: Monkey-patching gives you the flexibility to make changes to code at runtime, allowing you to 
experiment and try out new ideas without having to make permanent changes to the original code.

3. Power and control: Monkey-patching gives you a sense of power and control over the code, allowing you to tweak 
and customize its behavior to suit your needs.

4. Problem-solving: Monkey-patching can be a creative and satisfying way to solve complex problems or work around 
limitations in existing code.

Overall, the allure of monkey-patching in Python lies in its ability to empower developers to quickly and easily 
modify code, experiment with new ideas, and solve problems in a flexible and creative way. However, it's important 
to use monkey-patching judiciously and be mindful of the potential drawbacks, such as increased complexity and 
potential for unintended consequences.

Here's the last JSON

In [7]:
last_json_file = sorted(os.listdir(jsons_dir))[-1]
with open(os.path.join(jsons_dir, last_json_file), "r") as f:
    last_inputs = json.load(f)
print(last_inputs)

{
    'messages': [{'content': 'Why am I addicted to monkey-patching in Python?', 'role': 'user'}],
    'model': 'gpt-3.5-turbo',
    'stream': False,
    'n': 1,
    'temperature': 0.7
}

Or the lazy way / don't eagerly save JSONs, just add to a dict

In [8]:
with save_chat_create_inputs_as_dict(llm.client) as timestamp_to_kwargs:
    llm.invoke(
        "These LLM calls are independent. The next section has a more complicated demo."
    )
    response = llm.invoke(
        "Is How To with John Wilson an accurate portrayal of New York?"
    )

print(response.content)

It is difficult to say whether How To with John Wilson is an accurate portrayal of New York as it is a subjective 
and comedic documentary series. While the show does offer a unique and sometimes quirky perspective on life in the 
city, it may not capture the full diversity and complexity of New York. It is best to view the show as a creative 
and entertaining exploration of the city rather than a strictly accurate portrayal.

Let's see what's in `timestamp_to_kwargs`. There were 2 calls to the LLM, so there
should be 2 key-value pairs.

In [9]:
print(len(timestamp_to_kwargs))

2

In [10]:
print(timestamp_to_kwargs)

{
    '2024-04-03_17-09-54-148389': {
        'messages': [
            {
                'content': 'These LLM calls are independent. The next section has a more complicated demo.',
                'role': 'user'
            }
        ],
        'model': 'gpt-3.5-turbo',
        'stream': False,
        'n': 1,
        'temperature': 0.7
    },
    '2024-04-03_17-09-55-155838': {
        'messages': [
            {'content': 'Is How To with John Wilson an accurate portrayal of New York?', 'role': 'user'}
        ],
        'model': 'gpt-3.5-turbo',
        'stream': False,
        'n': 1,
        'temperature': 0.7
    }
}

We can modify `timestamp_to_kwargs` by passing it back in to a new context

In [11]:
with save_chat_create_inputs_as_dict(llm.client, existing_store=timestamp_to_kwargs):
    _ = llm.invoke("3. what am i doing")

We should now have 2 + 1 = 3 key-value pairs

In [12]:
print(len(timestamp_to_kwargs))

3

In [13]:
print(timestamp_to_kwargs)

{
    '2024-04-03_17-09-54-148389': {
        'messages': [
            {
                'content': 'These LLM calls are independent. The next section has a more complicated demo.',
                'role': 'user'
            }
        ],
        'model': 'gpt-3.5-turbo',
        'stream': False,
        'n': 1,
        'temperature': 0.7
    },
    '2024-04-03_17-09-55-155838': {
        'messages': [
            {'content': 'Is How To with John Wilson an accurate portrayal of New York?', 'role': 'user'}
        ],
        'model': 'gpt-3.5-turbo',
        'stream': False,
        'n': 1,
        'temperature': 0.7
    },
    '2024-04-03_17-10-01-103240': {
        'messages': [{'content': '3. what am i doing', 'role': 'user'}],
        'model': 'gpt-3.5-turbo',
        'stream': False,
        'n': 1,
        'temperature': 0.7
    }
}

# Demo w/ CoT and a tool

The first cell is just setup. It's pulled from
https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/07-langchain-tools.ipynb

In [14]:
from math import pi
from typing import Union

from langchain.tools import BaseTool
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.agents import initialize_agent


class CircumferenceTool(BaseTool):
    name = "Circumference calculator"
    description = "use this tool when you need to calculate a circumference using the radius of a circle"

    def _run(self, radius: Union[int, float]):
        if isinstance(radius, str):
            # stupid thing. I checked that this happens in the raw demo
            radius = radius.lstrip("radius=: ")
        return float(radius) * 2.0 * pi


OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY") or "OPENAI_API_KEY"

# initialize LLM (we use ChatOpenAI because we'll later define a `chat` agent)
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, temperature=0, model_name="gpt-3.5-turbo"
)
# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)


tools = [CircumferenceTool()]

# initialize agent with tools
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method="generate",
    memory=conversational_memory,
)

/Users/kdubey/Envs/base/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


If running the next cell raises something like this:

```python
ValueError: could not convert string to float: 'meter / 2; radius = 4 / 2; radius = 2'
```

please run it again. I don't think it's related to the patcher b/c it happens in naked
calls too.

In [15]:
jsons_dir = "temp"  # make this on your own
with save_chat_create_inputs_as_jsons(llm.client, jsons_dir):
    agent(
        "I have a circle with diameter 4. First calculate its radius. "
        "That'll be your first observation. "
        "In a new observation, calculate its circumference."
    )

/Users/kdubey/Envs/base/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
```json
{
    "action": "Circumference calculator",
    "action_input": "radius=2"
}
```
Observation: 12.566370614359172
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The circumference of the circle with a radius of 2 is approximately 12.57 units."
}
```

> Finished chain.


Here's what the last JSON looks like

In [16]:
_json_files = os.listdir(jsons_dir)
_num_jsons = len(_json_files)

last_json_file = sorted(_json_files)[-1]
with open(os.path.join(jsons_dir, last_json_file), "r") as f:
    last_inputs = json.load(f)
print(last_inputs)

{
    'messages': [
        {
            'content': 'Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able 
to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and 
discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on 
the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are 
coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities
are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to 
provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to 
generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations
and descriptions on a wide range of topics.\n\nOverall, Assistant is a powerful system that can help with a wide 
range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with 
a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.',
            'role': 'system'
        },
        {
            'content': 'TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be 
helpful in answering the users original question. The tools the human can use are:\n\n> Circumference calculator: 
use this tool when you need to calculate a circumference using the radius of a circle\n\nRESPONSE FORMAT 
INSTRUCTIONS\n----------------------------\n\nWhen responding to me, please output a response in one of two 
formats:\n\n**Option 1:**\nUse this if you want the human to use a tool.\nMarkdown code snippet formatted in the 
following schema:\n\n```json\n{\n    "action": string, \\\\ The action to take. Must be one of Circumference 
calculator\n    "action_input": string \\\\ The input to the action\n}\n```\n\n**Option #2:**\nUse this if you want
to respond directly to the human. Markdown code snippet formatted in the following schema:\n\n```json\n{\n    
"action": "Final Answer",\n    "action_input": string \\\\ You should put what you want to return to use 
here\n}\n```\n\nUSER\'S INPUT\n--------------------\nHere is the user\'s input (remember to respond with a markdown
code snippet of a json blob with a single action, and NOTHING else):\n\nI have a circle with diameter 4. First 
calculate its radius. That\'ll be your first observation. In a new observation, calculate its circumference.',
            'role': 'user'
        },
        {
            'content': '```json\n{\n    "action": "Circumference calculator",\n    "action_input": 
"radius=2"\n}\n```',
            'role': 'assistant'
        },
        {
            'content': "TOOL RESPONSE: \n---------------------\n12.566370614359172\n\nUSER'S 
INPUT\n--------------------\n\nOkay, so what is the response to my last comment? If using information obtained from
the tools you must mention it explicitly without mentioning the tool names - I have forgotten all TOOL RESPONSES! 
Remember to respond with a markdown code snippet of a json blob with a single action, and NOTHING else.",
            'role': 'user'
        }
    ],
    'model': 'gpt-3.5-turbo',
    'stream': False,
    'n': 1,
    'temperature': 0.0,
    'stop': ['\nObservation:', '\n\tObservation:']
}

If the context manager isn't used after it was used, verify that no extra JSONs are
saved, i.e., we're back to using the old `create` method

In [17]:
agent(
    "I have a circle with diameter 4. First calculate its radius. "
    "That'll be your first observation. "
    "In a new observation, calculate its circumference."
)



> Entering new AgentExecutor chain...
```json
{
    "action": "Circumference calculator",
    "action_input": "2"
}
```
Observation: 12.566370614359172
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The circumference of the circle with a radius of 2 is approximately 12.57 units."
}
```

> Finished chain.


{'input': "I have a circle with diameter 4. First calculate its radius. That'll be your first observation. In a new observation, calculate its circumference.",
 'chat_history': [HumanMessage(content="I have a circle with diameter 4. First calculate its radius. That'll be your first observation. In a new observation, calculate its circumference."),
  AIMessage(content='The circumference of the circle with a radius of 2 is approximately 12.57 units.')],
 'output': 'The circumference of the circle with a radius of 2 is approximately 12.57 units.'}

In [18]:
assert len(os.listdir(jsons_dir)) == _num_jsons

# Attempt to also save streamed outputs

I don't love this solution b/c it ends up modifying the return type, so any previous
`isinstance` or `issubclass` checks will return `False`. I think that's a dealbreaker,
but maybe there are some ducktype-pilled Pythonistas out there that are fine w/ it

In [19]:
# from functools import partial
from typing import Any, Callable, Generic, Iterable, Iterator, TypeVar

import openai

In [20]:
_T = TypeVar("_T")


def _generator_writer(
    generator: Iterable[_T], side_effect: Callable[[_T], Any]
) -> Generator[_T, None, None]:
    class _GeneratorWriter(Generic[_T]):
        """
        Proxy generator which also writes to `store`. 
        """

        def __iter__(self) -> Iterator[_T]:
            return self

        def __next__(self) -> _T:
            intermediate_output = next(generator)
            side_effect(intermediate_output)
            return intermediate_output

        def __getattr__(self, attr: str):
            return getattr(generator, attr)

    # TODO: dynamically subclass `generator.__class__`. Doing it in the declaration or
    # via _GeneratorWriter = type(...) messes up the __init__. Below is a failed attempt
    # https://bugs.python.org/issue672115
    # _GeneratorWriter.__bases__ += (generator.__class__,)
    return _GeneratorWriter()

In [21]:
def store_output(store: list, output: Any) -> None:
    store.append(output)

In [22]:
generator = reversed("hello")

character_store = []
side_effect = partial(store_output, character_store)

generator_which_writes = _generator_writer(generator, side_effect)

In [23]:
for char in generator_which_writes:
    print(char)

o

l

l

e

h

In [24]:
character_store

['o', 'l', 'l', 'e', 'h']

Now for a real demo

In [25]:
client = openai.OpenAI()

token_stream = client.chat.completions.create(
    messages=[{"role": "user", "content": "reply hello"}],
    model="gpt-3.5-turbo",
    stream=True,
)

chat_completion_chunk_store = []
generator_which_writes = _generator_writer(
    token_stream, partial(store_output, chat_completion_chunk_store)
)

In [26]:
for chunk in generator_which_writes:
    print(chunk.choices[0].delta.content)

Hello

!

How

can

I

assist

you

today

?

None

In [27]:
print(chat_completion_chunk_store)

[
    ChatCompletionChunk(
        id='chatcmpl-9A4yTfFJMr19mX0blaJy0DwhCKYXA',
        choices=[
            Choice(
                delta=ChoiceDelta(content='', function_call=None, role='assistant', tool_calls=None),
                finish_reason=None,
                index=0,
                logprobs=None
            )
        ],
        created=1712189429,
        model='gpt-3.5-turbo-0125',
        object='chat.completion.chunk',
        system_fingerprint='fp_b28b39ffa8'
    ),
    ChatCompletionChunk(
        id='chatcmpl-9A4yTfFJMr19mX0blaJy0DwhCKYXA',
        choices=[
            Choice(
                delta=ChoiceDelta(content='Hello', function_call=None, role=None, tool_calls=None),
                finish_reason=None,
                index=0,
                logprobs=None
            )
        ],
        created=1712189429,
        model='gpt-3.5-turbo-0125',
        object='chat.completion.chunk',
        system_fingerprint='fp_b28b39ffa8'
    ),
    ChatCompletionChunk(
        id='chatcmpl-9A4yTfFJMr19mX0blaJy0DwhCKYXA',
        choices=[
            Choice(
                delta=ChoiceDelta(content='!', function_call=None, role=None, tool_calls=None),
                finish_reason=None,
                index=0,
                logprobs=None
            )
        ],
        created=1712189429,
        model='gpt-3.5-turbo-0125',
        object='chat.completion.chunk',
        system_fingerprint='fp_b28b39ffa8'
    ),
    ChatCompletionChunk(
        id='chatcmpl-9A4yTfFJMr19mX0blaJy0DwhCKYXA',
        choices=[
            Choice(
                delta=ChoiceDelta(content=' How', function_call=None, role=None, tool_calls=None),
                finish_reason=None,
                index=0,
                logprobs=None
            )
        ],
        created=1712189429,
        model='gpt-3.5-turbo-0125',
        object='chat.completion.chunk',
        system_fingerprint='fp_b28b39ffa8'
    ),
    ChatCompletionChunk(
        id='chatcmpl-9A4yTfFJMr19mX0blaJy0DwhCKYXA',
        choices=[
            Choice(
                delta=ChoiceDelta(content=' can', function_call=None, role=None, tool_calls=None),
                finish_reason=None,
                index=0,
                logprobs=None
            )
        ],
        created=1712189429,
        model='gpt-3.5-turbo-0125',
        object='chat.completion.chunk',
        system_fingerprint='fp_b28b39ffa8'
    ),
    ChatCompletionChunk(
        id='chatcmpl-9A4yTfFJMr19mX0blaJy0DwhCKYXA',
        choices=[
            Choice(
                delta=ChoiceDelta(content=' I', function_call=None, role=None, tool_calls=None),
                finish_reason=None,
                index=0,
                logprobs=None
            )
        ],
        created=1712189429,
        model='gpt-3.5-turbo-0125',
        object='chat.completion.chunk',
        system_fingerprint='fp_b28b39ffa8'
    ),
    ChatCompletionChunk(
        id='chatcmpl-9A4yTfFJMr19mX0blaJy0DwhCKYXA',
        choices=[
            Choice(
                delta=ChoiceDelta(content=' assist', function_call=None, role=None, tool_calls=None),
                finish_reason=None,
                index=0,
                logprobs=None
            )
        ],
        created=1712189429,
        model='gpt-3.5-turbo-0125',
        object='chat.completion.chunk',
        system_fingerprint='fp_b28b39ffa8'
    ),
    ChatCompletionChunk(
        id='chatcmpl-9A4yTfFJMr19mX0blaJy0DwhCKYXA',
        choices=[
            Choice(
                delta=ChoiceDelta(content=' you', function_call=None, role=None, tool_calls=None),
                finish_reason=None,
                index=0,
                logprobs=None
            )
        ],
        created=1712189429,
        model='gpt-3.5-turbo-0125',
        object='chat.completion.chunk',
        system_fingerprint='fp_b28b39ffa8'
    ),
    ChatCompletionChunk(
        id='chatcmpl-9A4yTfFJMr19mX0blaJy0DwhCKYXA',
        choices=[
            Choice(
